In [ ]:
import open3d
import argparse
import sys
sys.path.append('.')
import os
import time
import h5py
import datetime
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn.parallel
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from data_utils.ModelNetDataLoader import ModelNetDataLoader, load_data, class_names
from pathlib import Path
from tqdm import tqdm_notebook as tqdm
from utils import test, save_checkpoint, select_avaliable, mkdir
from model.pointnet2 import PointNet2ClsMsg
from model.pointnet import PointNetCls, feature_transform_reguliarzer
import colorsys
from clf import parse_args, root

import my_log as log
from my_fn import *

args = parse_args([])
args.pretrain = 'experiment/weights/clf-pointnet-0.89730-0076.pth'
args.batch_size = 8
test_data, test_label = load_data(root, train = False)
testDataset = ModelNetDataLoader(test_data, test_label)
testDataLoader = torch.utils.data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False)

log.debug('Building Model',args.model_name)
if args.model_name == 'pointnet':
    num_class = 40
    model = PointNetCls(num_class,args.feature_transform).cuda()  
else:
    model = PointNet2ClsMsg().cuda()

if args.pretrain is None:
    log.err('No pretrain model')
else:
    log.debug('Loading pretrain model...')
    checkpoint = torch.load(args.pretrain)
    model.load_state_dict(checkpoint)
    model.eval()
    print('Done')

In [ ]:
def disp_pt_cloud(x,channal_first = True):
    cloud = open3d.geometry.PointCloud()
    cloud.points = open3d.utility.Vector3dVector(x)

    vis = open3d.visualization.VisualizerWithKeyCallback()
    vis.create_window(args.model_name, height=800, width=800, left=200, top=0)
    opt = vis.get_render_option().background_color = np.asarray([0, 0, 0])
    vis.add_geometry(cloud)
    vis.register_key_callback(32, lambda vis: exit())
    vis.run()
    vis.destroy_window()

epoches = 50
buf,buf2 = [],[]
for pts, gt in testDataLoader:
    gt = gt[:, 0].long().cuda()
    pts.requires_grad = False
    pts = pts.transpose(2, 1).cuda() # make channal first

    output, _ = model(pts)
    org_pred = output.data.max(1)[1]
    target = torch.from_numpy(np.array([0]* args.batch_size)).cuda()
    noise = torch.zeros_like(pts, requires_grad=True).cuda()
    opt = optim.Adam([noise], lr=0.05, betas=(0.9, 0.999))
    
    for i in tqdm(range(epoches)):        
        opt.zero_grad()
        pts_adv = pts + noise
        softmax, _ = model(pts_adv)
        loss_cross = nn.CrossEntropyLoss()(softmax, target)
        loss_chamfer = chamfer_batch(pts_adv.transpose(2, 1), pts.transpose(2, 1))
        loss = loss_cross*0.5 + loss_chamfer
        loss.backward()
        opt.step()
            
        output, _ = model(pts_adv)
        adv_chocie = output.data.max(1)[1]
        is_target = num((adv_chocie == target).float().mean())
        not_orginal = num((adv_chocie == org_pred).float().mean())
        buf.append([num(loss), num(loss_cross), num(loss_chamfer)])
        buf2.append([is_target, not_orginal])
        del output,adv_chocie,is_target,not_orginal
    break

fig = plt.figure(figsize=(12, 2), dpi=80, facecolor='w', edgecolor='k')
for i,title in enumerate(['loss sum', 'CrossEntropyLoss ', 'ChamferLoss']):
    ax = plt.subplot(1,4,i+1);
    plt.plot(np.array(buf)[:,i]);
    ax.title.set_text(title);
ax = plt.subplot(1,4,4);
plt.plot(np.array(buf2));
ax.title.set_text('is_target & not_orginal');
plt.show()

In [ ]:
idx = 6
disp_pt_cloud(num(pts.detach().transpose(2, 1)[idx]))
disp_pt_cloud(num(pts_adv.detach().transpose(2, 1)[idx]))

In [ ]:
aa, _ = model(pts)
bb, _ = model(pts_adv)
print('org:', np.array(class_names)[num(aa.data.max(1)[1])].tolist())
print('adv:',np.array(class_names)[num(bb.data.max(1)[1])].tolist())